<a href="https://colab.research.google.com/github/luisashimabucoro/NMA-Project-Code-Summarization/blob/main/Starter_NMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preliminaries**

In [46]:
!pip install transformers &> /dev/null

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import transformers
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader


In [47]:
# mount gdrive

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Basic EDA**

In [48]:
data_df = pd.read_json('/content/gdrive/Shareddrives/Dolma2/Datasets/python/train.jsonl', lines=True)

In [49]:
data_df.head() # samples

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,smdabdoub/phylotoast,phylotoast/util.py,split_phylogeny,"def split_phylogeny(p, level=""s""):\n """"""\n ...",python,"def split_phylogeny(p, level=""s""):\n """"""\n ...","[def, split_phylogeny, (, p, ,, level, =, ""s"",...",Return either the full or truncated version of...,"[Return, either, the, full, or, truncated, ver...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0...,train
1,smdabdoub/phylotoast,phylotoast/util.py,ensure_dir,"def ensure_dir(d):\n """"""\n Check to make...",python,"def ensure_dir(d):\n """"""\n Check to make...","[def, ensure_dir, (, d, ), :, if, not, os, ., ...",Check to make sure the supplied directory path...,"[Check, to, make, sure, the, supplied, directo...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0...,train
2,smdabdoub/phylotoast,phylotoast/util.py,file_handle,"def file_handle(fnh, mode=""rU""):\n """"""\n ...",python,"def file_handle(fnh, mode=""rU""):\n """"""\n ...","[def, file_handle, (, fnh, ,, mode, =, ""rU"", )...",Takes either a file path or an open file handl...,"[Takes, either, a, file, path, or, an, open, f...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0...,train
3,smdabdoub/phylotoast,phylotoast/util.py,gather_categories,"def gather_categories(imap, header, categories...",python,"def gather_categories(imap, header, categories...","[def, gather_categories, (, imap, ,, header, ,...",Find the user specified categories in the map ...,"[Find, the, user, specified, categories, in, t...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0...,train
4,smdabdoub/phylotoast,phylotoast/util.py,parse_unifrac,"def parse_unifrac(unifracFN):\n """"""\n Pa...",python,"def parse_unifrac(unifracFN):\n """"""\n Pa...","[def, parse_unifrac, (, unifracFN, ), :, with,...",Parses the unifrac results file into a diction...,"[Parses, the, unifrac, results, file, into, a,...",0b74ef171e6a84761710548501dfac71285a58a3,https://github.com/smdabdoub/phylotoast/blob/0...,train


In [50]:
data_df.loc[:,['code', 'docstring']].head(12) # removing extra columns


,code,docstring
0,"def split_phylogeny(p, level=""s""):\n """"""\n ...",Return either the full or truncated version of...
1,"def ensure_dir(d):\n """"""\n Check to make...",Check to make sure the supplied directory path...
2,"def file_handle(fnh, mode=""rU""):\n """"""\n ...",Takes either a file path or an open file handl...
3,"def gather_categories(imap, header, categories...",Find the user specified categories in the map ...
4,"def parse_unifrac(unifracFN):\n """"""\n Pa...",Parses the unifrac results file into a diction...
5,"def parse_unifrac_v1_8(unifrac, file_data):\n ...",Function to parse data from older version of u...
6,"def parse_unifrac_v1_9(unifrac, file_data):\n ...",Function to parse data from newer version of u...
7,"def color_mapping(sample_map, header, group_co...",Determine color-category mapping. If color_col...
8,"def rev_c(read):\n """"""\n return reverse ...",return reverse completment of read
9,"def shuffle_genome(genome, cat, fraction = flo...",randomly shuffle genome


**DataLoader**

In [51]:


class Code2DocDataset(Dataset):
  def __init__(self, df, is_test=False):
        self.df = df.reset_index(drop=True)
        self.is_test = is_test

  def __getitem__(self, idx):
    sample = self.df.loc[:,['code', 'docstring']]    
    return dict(sample.loc[idx]) # return dict

  def __len__(self):
    return len(self.df)


In [52]:
from torch.utils.data import Dataset, DataLoader

train_set = Code2DocDataset(data_df)
train_loader = DataLoader(train_set)

In [53]:
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer

config = RobertaConfig.from_pretrained("microsoft/codebert-base")
tokenizer = RobertaTokenizer.from_pretrained(
    "microsoft/codebert-base", do_lower_case=False
)

**Tokenizer**

In [54]:
tokenizer("Hello World! Valar Morghulis")

{'input_ids': [0, 31414, 623, 328, 3767, 271, 4266, 4147, 922, 354, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [55]:
for i in train_loader:
  print(i['docstring'])
  print(tokenizer(i['docstring']))
  print(i['code'])
  print(tokenizer(i['code']))
  break

['Return either the full or truncated version of a QIIME-formatted taxonomy string.\n\n    :type p: str\n    :param p: A QIIME-formatted taxonomy string: k__Foo; p__Bar; ...\n\n    :type level: str\n    :param level: The different level of identification are kingdom (k), phylum (p),\n                  class (c),order (o), family (f), genus (g) and species (s). If level is\n                  not provided, the default level of identification is species.\n\n    :rtype: str\n    :return: A QIIME-formatted taxonomy string up to the classification given\n            by param level.']
{'input_ids': [[0, 42555, 1169, 5, 455, 50, 43064, 1070, 1732, 9, 10, 1209, 100, 28417, 12, 3899, 19747, 629, 38217, 6755, 4, 50140, 1437, 1437, 1437, 4832, 12528, 181, 35, 7031, 50118, 1437, 1437, 1437, 4832, 46669, 181, 35, 83, 1209, 100, 28417, 12, 3899, 19747, 629, 38217, 6755, 35, 449, 30529, 597, 3036, 131, 181, 30529, 14507, 131, 1666, 50140, 1437, 1437, 1437, 4832, 12528, 672, 35, 7031, 50118, 1437, 1437

**Model**

In [56]:
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer

config = RobertaConfig.from_pretrained("microsoft/codebert-base")
tokenizer = RobertaTokenizer.from_pretrained(
    "microsoft/codebert-base", do_lower_case=False
)